In [20]:
import os
import glob
import pandas as pd

from utils import (
    errors_idxs,
    top_k,
    calc_differentially_expressed_genes,
    calc_res
)

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

##### cohort data
All data files should have binary group column (`True` for the positive class and `False` for the negative)

In [21]:
path_to_cohort_data = 'data/cohort_data.csv'
index_col = 'Unnamed: 0'
data = pd.read_csv(path_to_cohort_data, index_col=index_col)

##### read gt data

All gt datasets should be inside the data folder in this format:

gt_0.csv <br>
gt_1.csv <br>
. <br>
. <br>
.

In [22]:
files = glob.glob('data/gt*.csv')
dfs = {}
cols = {}
index_col = 'Unnamed: 0'
for f in files:
    number = os.path.splitext(f)[0].split('_')[1]
    dfs[int(number)] = pd.read_csv(f, index_col=index_col)

### Calculations

The function `calc_differentially_expressed_genes` returns a report with the following fileds:
1. 'n_genes': number of differentially expressed genes genes in each label swap
1. 'pvalue_original_sorted': WRS p-value for each gene sorted in increasing order
1. 'sorted_genes': the complete gene list sorted according to thier p-values ('pvalue_original_sorted')
1. 'original_list': differentially expressed genes as calculated without label swaps
1. 'gene_list': the RDEG set

##### cohort data calculations

In [25]:
cohort_report = calc_differentially_expressed_genes(data, 0.001, 'group', n_changes=1, alternative='two-sided')

100%|██████████| 104/104 [00:00<00:00, 619.10it/s]


##### gt calculations

In [26]:
GT_set = {}
for gt_number in dfs:
    report = calc_differentially_expressed_genes(dfs[gt_number], 0.001, 'group', n_changes=1, 
                                                 alternative='two-sided')
    if len(GT_set):
        GT_set = GT_set.intersection(report['original_list'])
    else:
        GT_set = set(report['original_list'])

100%|██████████| 93/93 [00:00<00:00, 800.38it/s]


##### calc results

In [27]:
TP = len(GT_set.intersection(set(cohort_report['original_list'])))
calc_res('Original - GT FDR intersection', len(cohort_report['sorted_genes']),
         P=len(GT_set), predicted_P=len(set(cohort_report['original_list'])), TP=TP)
print('----------')
TP = len(GT_set.intersection(set(cohort_report['gene_list'])))
calc_res('RDEG - GT FDR intersection', len(cohort_report['sorted_genes']),
         P=len(GT_set), predicted_P=len(set(cohort_report['gene_list'])), TP=TP)
print('='*20)

Original - GT FDR intersection
Predicted Positive: 316
TP: 94 FP: 222 TN: 9178 FN: 38
Precision: 0.297
Recall: 0.712
----------
RDEG - GT FDR intersection
Predicted Positive: 138
TP: 71 FP: 67 TN: 9333 FN: 61
Precision: 0.514
Recall: 0.538
